Section	A:	Problem	Statement	–	Enhancing	Neural	Network	Performance	with	Particle	Swarm	Optimization	

In [1]:
#Download and Load Dataset

# Step 1: Install required libraries
!pip install pyswarm

# Step 2: Import packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from pyswarm import pso  # Particle Swarm Optimization

# Step 3: Load dataset
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
data = pd.read_csv(url, sep=';')

# Step 4: Preprocess dataset
X = data.drop('quality', axis=1).values
y = data['quality'].values
y = (y >= 6).astype(int)  # Convert to binary classification: good (1) vs bad (0)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Traditional Neural Network with Manual Tuning

# Build basic NN
def build_nn_model(input_dim, hidden_units=16, learning_rate=0.01):
    model = Sequential([
        Dense(hidden_units, input_dim=input_dim, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train and evaluate
model = build_nn_model(X_train.shape[1], hidden_units=32)
history = model.fit(X_train, y_train, epochs=20, verbose=0, batch_size=32)
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Manual NN Accuracy: {accuracy:.4f}")



C:\Users\patel\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Manual NN Accuracy: 0.7406


In [3]:
# PSO-Optimized Neural Network

# Objective function for PSO: minimize negative accuracy
def objective_function(params):
    hidden_units = int(params[0])
    learning_rate = params[1]
    
    model = Sequential([
        Dense(hidden_units, input_dim=X_train.shape[1], activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=15, verbose=0, batch_size=32)
    _, acc = model.evaluate(X_test, y_test, verbose=0)
    
    return -acc  # PSO minimizes, so we negate accuracy

# Search space: [hidden_units, learning_rate]
lb = [10, 0.0001]
ub = [100, 0.1]

# Run PSO
best_params, best_score = pso(objective_function, lb, ub, swarmsize=5, maxiter=3)
print(f"Best Parameters from PSO: Hidden Units = {int(best_params[0])}, Learning Rate = {best_params[1]:.5f}")
print(f"PSO Optimized Accuracy: {-best_score:.4f}")


Stopping search: maximum iterations reached --> 3
Best Parameters from PSO: Hidden Units = 51, Learning Rate = 0.00010
PSO Optimized Accuracy: 0.7469


In [4]:
# Final Comparison
print("\n--- Final Comparison ---")
print(f"Manual Tuned Accuracy: {accuracy:.4f}")
print(f"PSO Optimized Accuracy: {-best_score:.4f}")



--- Final Comparison ---
Manual Tuned Accuracy: 0.7406
PSO Optimized Accuracy: 0.7469


 Section	B:	Task	Description	

In [1]:
# Install Required Packages
!pip install scikit-learn keras pyswarm



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Load and Preprocess the Dataset
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Load a small subset of the 20 newsgroups dataset for speed
categories = ['rec.autos', 'sci.space', 'talk.politics.misc', 'comp.graphics']
data = fetch_20newsgroups(subset='all', categories=categories, remove=('headers', 'footers', 'quotes'))

# Features and Labels
X = data.data
y = data.target

# Text vectorization
vectorizer = TfidfVectorizer(max_features=2000)  # Limit to 2000 features for speed
X_vect = vectorizer.fit_transform(X).toarray()

# Encode labels
le = LabelEncoder()
y_enc = le.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_vect, y_enc, test_size=0.2, random_state=42)


In [3]:
# Build and Train Baseline Neural Network

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score

# One-hot encode labels for classification
y_train_cat = to_categorical(y_train, num_classes=4)
y_test_cat = to_categorical(y_test, num_classes=4)

# Function to build the model
def build_nn(input_dim, hidden_units=64, lr=0.001):
    model = Sequential([
        Dense(hidden_units, activation='relu', input_shape=(input_dim,)),
        Dense(4, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Train with default hyperparameters
model = build_nn(X_train.shape[1], hidden_units=64, lr=0.001)
model.fit(X_train, y_train_cat, epochs=5, batch_size=32, verbose=0)

# Evaluate
loss, acc = model.evaluate(X_test, y_test_cat, verbose=0)
print(f"Baseline Accuracy: {acc:.4f}")


C:\Users\patel\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Baseline Accuracy: 0.8617


In [4]:
# PSO to Optimize Neural Network Hyperparameters
from pyswarm import pso

# Objective function for PSO
def pso_objective(params):
    hidden_units = int(params[0])
    lr = params[1]

    model = build_nn(X_train.shape[1], hidden_units=hidden_units, lr=lr)
    model.fit(X_train, y_train_cat, epochs=3, batch_size=32, verbose=0)
    loss, acc = model.evaluate(X_test, y_test_cat, verbose=0)
    
    return -acc  # PSO minimizes this

# Define hyperparameter bounds: [hidden_units, learning_rate]
lb = [16, 0.0001]
ub = [128, 0.01]

# Run PSO (small particles and iterations to keep runtime low)
best_params, best_score = pso(pso_objective, lb, ub, swarmsize=5, maxiter=3)

print(f"\nPSO Best Hidden Units: {int(best_params[0])}")
print(f"PSO Best Learning Rate: {best_params[1]:.5f}")
print(f"PSO Optimized Accuracy: {-best_score:.4f}")


Stopping search: maximum iterations reached --> 3

PSO Best Hidden Units: 123
PSO Best Learning Rate: 0.00109
PSO Optimized Accuracy: 0.8644


In [5]:
# Final Comparison and Analysis
print("\n--- Final Comparison ---")
print(f"Traditional NN Accuracy: {acc:.4f}")
print(f"PSO-Optimized NN Accuracy: {-best_score:.4f}")



--- Final Comparison ---
Traditional NN Accuracy: 0.8617
PSO-Optimized NN Accuracy: 0.8644
